In [1]:
from argparse import ArgumentParser
from dataclasses import dataclass
import os
from typing import List, Dict

import yaml

# from task_functions import parse, preprocess, featurize_tokens, featurize_cells, train, evaluate
from fluidml.common import Resource
from fluidml.flow import Flow
from fluidml.flow import GridTaskSpec, TaskSpec
from fluidml.swarm import Swarm
from fluidml.storage import LocalFileStore, MongoDBStore, InMemoryStore

In [2]:
def parse(results: Dict, in_dir: str):
    return {}


def preprocess(results: Dict, pipeline: List[str], abc: List[int]):
    return {}

def featurize_tokens(results: Dict, type_: str, batch_size: int):
    return {}


def featurize_cells(results: Dict, type_: str, batch_size: int):
    return {}


def train(results: Dict, model, dataloader, evaluator, optimizer, num_epochs):
    return {'score': 2.}  # 'score': 2.


def evaluate(results: Dict, metric: str):
    print(results)
    return {}

In [9]:
CURRENT_DIR = os.getcwd()
base_dir = os.path.join(CURRENT_DIR, 'experiments')

In [4]:
parse_task = GridTaskSpec(task=parse, gs_config={"in_dir": "/some/dir"})
preprocess_task = GridTaskSpec(task=preprocess, gs_config={"pipeline": ['a', 'b'], "abc": [1,2]})
featurize_tokens_task = GridTaskSpec(task=featurize_tokens, gs_config={"type_": "flair", 'batch_size': 2})
featurize_cells_task = GridTaskSpec(task=featurize_cells, gs_config={"type_": "glove", "batch_size": 4})
train_task = GridTaskSpec(task=train, gs_config={"model": "mlp", "dataloader": "x", "evaluator": "y", "optimizer": "adam", "num_epochs": 10})
evaluate_task = TaskSpec(task=evaluate, reduce=True, task_kwargs={"metric": "accuracy"})

In [5]:
# dependencies between tasks
preprocess_task.requires([parse_task])
featurize_tokens_task.requires([preprocess_task])
featurize_cells_task.requires([preprocess_task])
train_task.requires([featurize_tokens_task, featurize_cells_task])
evaluate_task.requires([train_task])

In [6]:
# all tasks
tasks = [parse_task,
         preprocess_task,
         featurize_tokens_task,
         featurize_cells_task,
         train_task,
         evaluate_task]

In [10]:
results_store = LocalFileStore(base_dir=base_dir)

In [11]:
# run tasks in parallel (GridTaskSpecs are expanded based on grid search arguments)
with Swarm(n_dolphins=4, results_store=results_store) as swarm:
    flow = Flow(swarm=swarm, force='all')
    results = flow.run(tasks)
    # print(results)
    # print('bla')